In [ ]:
from ipywidgets import Output

out = Output()

In [ ]:
import pandas
from astropy.table import Table
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy import units as u
from astropy.nddata import CCDData
from pathlib import Path
from glowing_waffles.differential_photometry import *
from astrowidgets import ImageWidget, __version__
from glowing_waffles.photometry import *
import numpy as np
from comparison_functions import *
import astrowidgets
from seeing_profile_functions import *

In [ ]:
print(astrowidgets.__version__)

In [ ]:
directory_with_images = '/Users/madelyn/Desktop/2019-10-25-reduced'
#This could be any image from roughly the middle of the sequence.
sample_image_for_finding_stars = 'TIC_167031605.01-S001-R001-C001-rp.fit'
object_of_interest = ''

In [ ]:
df, RD, ra, dec, RD['coords'] = read_file('/Users/madelyn/Documents/Apprenticeship/gaia_stars_1573775995.radec')

In [ ]:
ccd, ccdx, ccdy, vsx, ra, dec, vsx['coords'], vsx_names = set_up('', directory_with_images, sample_image_for_finding_stars)

In [ ]:
apass, v_angle, RD_angle = match(ccd, RD, vsx)

In [ ]:
Cmag = 9.9

In [ ]:
apass_good_coord, good_stars = mag_scale(Cmag, apass, v_angle, RD_angle)

In [ ]:
ent = in_field(apass_good_coord, ccd, ccdx, ccdy, apass, good_stars)

In [ ]:
iw = ImageWidget()
iw

In [ ]:
make_markers(iw, directory_with_images, sample_image_for_finding_stars, RD, vsx, ent, '')

In [ ]:
bind_map = iw._viewer.get_bindmap()
gvc = iw._viewer.get_canvas()
bind_map.map_event(None, ('shift',), 'ms_left', 'cursor')
gvc.add_callback('cursor-down', wrap(iw, out))

In [ ]:
out

In [ ]:
pic_name = 'kelt16-apertures-x'

In [ ]:
iw.save(pic_name + '.png')

In [ ]:
all_table = iw.get_markers(marker_name='all')

In [ ]:
f = all_table['marker name'] == 'elim'

In [ ]:
good_table = all_table[~f]

In [ ]:
all_table = iw.get_markers(marker_name = 'all')
elims = np.array([name.startswith('elim') for name in all_table['marker name']])
elim_table = all_table[elims]
comp_table = all_table[~elims]

In [ ]:
comp_table
elim_table

In [ ]:
index, d2d, d3d = elim_table['coord'].match_to_catalog_sky(comp_table['coord'])
comp_table.remove_rows(index)

In [ ]:
comp_table

In [ ]:
comp_table.write('comp_locations.fits')

In [ ]:
iw.stop_marking()